# OSNAP line Lagrangian particle tracking investigation of the cold/fresh blob


### The aim here is an investigation of the source of the cold/fresh blob which appeared in the near-surface Iceland Basin from about 2014. There are a couple of published explanations for the feature: 

- reduced heat/salt transport across RAPID array over several years propagating northwards (Bryden et al. 2020?) or 
- increased contribution from Labrador Sea water flowing into subpolar North Atlantic rather than southwards (Holliday et al. 2020?). Driven I think by specific,  more intermittent, weather near Flemish Cap (though I need to read this again...)

The technique is to track particles back from the OSNAP line, initially for 2 years. Each particle has an associated transport normal to OSNAP, which I use to estimate the contribution to volumes/heat/salt crossing OSNAP northwards from the two main routes - Gulf Stream and Labrador Sea. And see how that changes over the years from 2006 to 2017.

The main conclusion is that from about 2012 Labrador Sea becomes a more important source of surface waters to the Iceland Basin, but there is also a reduction in the temperature and salinity of the water from the Gulf Stream. So probably a balance of both. **But I need to run the particles back further - 2 years is not long enough to pinpoint the pathway for most particles.**

## Technical preamble

In [ ]:
# import matplotlib.colors as colors
import numpy as np
from pathlib import Path
import matplotlib.pyplot as plt
import xarray as xr
from datetime import datetime, timedelta
import seaborn as sns
# from matplotlib.colors import ListedColormap
import cmocean as co
import pandas as pd
import matplotlib.dates as mdates
import cartopy.crs as ccrs
import cartopy
import seawater as sw

from matplotlib import colors as c
from matplotlib import ticker
from xhistogram.xarray import histogram

sns.set(style="darkgrid")

xr.set_options(keep_attrs=True)
np.warnings.filterwarnings('ignore')

sns.set_palette("colorblind")
xr.set_options(keep_attrs=True);


## Set up paths and read in trajectory data

In [ ]:
## Parameters
# Project path
project_path = Path.cwd() / '..' / '..' 
project_path = project_path.resolve()

# Parcels track data file
path_data_tracks = Path('data/processed/tracks/osnap/') 
filename_tracks = [
                   'tracks_osnap_backward_200607_N2931_D1460_Rnd123.nc',
                   'tracks_osnap_backward_200707_N2931_D1460_Rnd123.nc',
                   'tracks_osnap_backward_200807_N2931_D1460_Rnd123.nc',
                   'tracks_osnap_backward_200907_N2931_D1460_Rnd123.nc',
                   'tracks_osnap_backward_201007_N2931_D1460_Rnd123.nc',
                   'tracks_osnap_backward_201107_N2931_D1460_Rnd123.nc',
                   'tracks_osnap_backward_201207_N2931_D1460_Rnd123.nc',
                   'tracks_osnap_backward_201307_N2931_D1460_Rnd123.nc',
                   'tracks_osnap_backward_201407_N2931_D1460_Rnd123.nc',
                   'tracks_osnap_backward_201507_N2931_D1460_Rnd123.nc',
                   'tracks_osnap_backward_201607_N2931_D1460_Rnd123.nc',
                   'tracks_osnap_backward_201707_N2931_D1460_Rnd123.nc',
                   'tracks_osnap_backward_200601_N2931_D1460_Rnd123.nc',
                   'tracks_osnap_backward_200701_N2931_D1460_Rnd123.nc',
                   'tracks_osnap_backward_200801_N2931_D1460_Rnd123.nc',
                   'tracks_osnap_backward_200901_N2931_D1460_Rnd123.nc',
                   'tracks_osnap_backward_201001_N2931_D1460_Rnd123.nc',
                   'tracks_osnap_backward_201101_N2931_D1460_Rnd123.nc',
                   'tracks_osnap_backward_201201_N2931_D1460_Rnd123.nc',
                   'tracks_osnap_backward_201301_N2931_D1460_Rnd123.nc',
                   'tracks_osnap_backward_201401_N2931_D1460_Rnd123.nc',
                   'tracks_osnap_backward_201501_N2931_D1460_Rnd123.nc',
                   'tracks_osnap_backward_201601_N2931_D1460_Rnd123.nc',
                   'tracks_osnap_backward_201701_N2931_D1460_Rnd123.nc'
]

# model mask file
data_path = Path("data/external/iAtlantic/")
experiment_name = "VIKING20X.L46-KKG36107B"
mesh_mask_file = project_path / data_path / "mask" / experiment_name / "1_mesh_mask.nc"

#section lonlat file
sectionPath = Path('data/external/sections/')
sectionFilename = 'osnap_pos_wp.txt'
sectionname = 'osnap'
gsrsectionFilename = 'gsr_pos_wp.txt'

degree2km = 1.852*60.0


# some transport values specific to osnap runs
# randomly seeded 39995 particles, 19886 were in ocean points (the rest were land)

# osnap_section_length = 3594572.87839    # m
# osnap_section_depth = 4000 # m over which particles launched

# osnap_section_ocean_area = osnap_section_length * osnap_section_depth * 19886.0 / 39995.0

# particle_section_area = osnap_section_length * osnap_section_depth / 39995.0

osnap_section_length = 3594572.87839    # m
osnap_subsection_length = 1121724.76238   # m
osnap_section_depth = 4000 # m over which particles launched
osnap_subsection_depth = 500 # m over which particles launched

osnap_subsection_ocean_area = osnap_subsection_length * osnap_subsection_depth * 2931.0 / 3119.0

particle_section_area = osnap_subsection_length * osnap_subsection_depth / 3119.0


## Load data

### mesh and masks

In [ ]:
mesh_mask = xr.open_dataset(mesh_mask_file)
mesh_mask = mesh_mask.squeeze()
mesh_mask = mesh_mask.set_coords(["nav_lon", "nav_lat", "nav_lev"])

bathy = mesh_mask.mbathy.rename("number of water filled points")

depth = (mesh_mask.e3t_0 * mesh_mask.tmask).sum("z")
# display(mesh_mask)

### section position data

In [ ]:
lonlat = xr.Dataset(pd.read_csv(project_path / sectionPath / sectionFilename,delim_whitespace=True))
lonlat['lon'] *= -1.0

In [ ]:
lonlat.lon.attrs['long_name']='Longitude'
lonlat.lat.attrs['long_name']='Latitude'
lonlat.lon.attrs['standard_name']='longitude'
lonlat.lat.attrs['standard_name']='latitude'
lonlat.lon.attrs['units']='degrees_east'
lonlat.lat.attrs['units']='degrees_north'

lonlat2mean= lonlat.rolling({'dim_0':2}).mean()

lonlatdiff = (lonlat.diff('dim_0'))

lonlatdiff = lonlatdiff.assign({'y':lonlatdiff['lat']*degree2km})
lonlatdiff = lonlatdiff.assign({'x':lonlatdiff['lon']*degree2km*np.cos(np.radians(lonlat2mean.lat.data[1:]))})
lonlatdiff=lonlatdiff.assign({'length':np.sqrt(lonlatdiff['x']**2+lonlatdiff['y']**2)})
lonlatdiff=lonlatdiff.assign({'costheta':lonlatdiff['x']/lonlatdiff['length']})
lonlatdiff=lonlatdiff.assign({'sintheta':lonlatdiff['y']/lonlatdiff['length']})

total_length = lonlatdiff.length.sum().data 
total_osnap_length = lonlatdiff.length[0:12].sum().data;  # exclude section across UK - just there for testing north/south

length_west = xr.concat((xr.DataArray([0],dims=("dim_0"),coords={"dim_0": [0]}),lonlatdiff.length.cumsum()),dim='dim_0')

### tracks

In [ ]:
lonlatdiff.length.cumsum().data[9] - lonlatdiff.length.cumsum().data[5]

In [ ]:
ds=[]
for filename in filename_tracks:
    ds.append(xr.open_dataset(project_path / path_data_tracks / filename))
# ds.isel(obs=0).z.max()

#### Subset tracks by OSNAP line cross longitude and depth range

In [ ]:
lonRange=[-30,-19]
depthRange=[0,500]

In [ ]:
for i in range(len(ds)):
    ds[i] = ds[i].where(ds[i].isel(obs=0).lon > lonRange[0]).where(ds[i].isel(obs=0).lon < lonRange[1])
    ds[i] = ds[i].where(ds[i].isel(obs=0).z > depthRange[0]).where(ds[i].isel(obs=0).z < depthRange[1])
    ds[i] = ds[i].dropna('traj', how='all')
    

#### Velocity conversions from degrees lat/lon per second to m/s

In [ ]:
for i in range(len(ds)):
    ds[i]=ds[i].assign({'uvel_ms':ds[i].uvel * degree2km * 1000.0 * np.cos(np.radians(ds[i].lat))})
    ds[i]=ds[i].assign({'vvel_ms':ds[i].vvel * degree2km * 1000.0})

#### Find initial velocities normal to the section

In [ ]:
ds_init=[]

for dsi in ds:
    ds_temp = dsi.isel(obs=0)
    
    ds_init.append(ds_temp)
    
for i in range(len(ds_init)):
    ds_init[i] = ds_init[i].assign({'section_index':xr.DataArray(np.searchsorted(lonlat.lon,ds_temp.lon)-1,dims='traj')})
    ds_init[i] = ds_init[i].assign({'u_normal':ds_init[i].vvel_ms * lonlatdiff.costheta[ds_init[i].section_index].data - ds_init[i].uvel_ms * lonlatdiff.sintheta[ds_init[i].section_index].data})
    ds_init[i] = ds_init[i].assign({'u_along':ds_init[i].vvel_ms * lonlatdiff.sintheta[ds_init[i].section_index].data + ds_init[i].uvel_ms * lonlatdiff.costheta[ds_init[i].section_index].data})
    ds_init[i] = ds_init[i].assign({'vol_trans_normal':ds_init[i].u_normal * particle_section_area/1.0e06})


#### Find along-section distances of initial points

In [ ]:
for i in range(len(ds_init)):
    ds_init[i] = ds_init[i].assign({'x':xr.DataArray(length_west[ds_init[i].section_index] + lonlatdiff.length[ds_init[i].section_index]*
                              (ds_init[i].lon - lonlat.lon[ds_init[i].section_index])/lonlatdiff.lon[ds_init[i].section_index],dims='traj')})

### temperature and salt transports along track

In [ ]:
for i in range(len(ds)):
    ds[i]=ds[i].assign({'temp_transport':ds[i].temp * ds_init[i].vol_trans_normal})
    ds[i]=ds[i].assign({'salt_transport':ds[i].salt * ds_init[i].vol_trans_normal})
    ds[i]=ds[i].assign({'vol_transport':ds_init[i].vol_trans_normal})


In [ ]:
ds[1]

## Plot section

In [ ]:
sns.set(style="whitegrid")
central_lon, central_lat = -30, 55
fig, ax = plt.subplots(subplot_kw={'projection': ccrs.Orthographic(central_lon, central_lat)})
extent = [-60, 0, 40, 70]
ax.set_extent(extent)
ax.gridlines()
ax.coastlines(resolution='50m')

lonlat.plot.scatter(ax=ax,transform=ccrs.PlateCarree(),x='lon',y='lat')
lonlat2mean.plot.scatter(ax=ax,transform=ccrs.PlateCarree(),x='lon',y='lat');

## Find track source regions

In [ ]:
def apply_left_of_line(ds, lon_1, lon_2, lat_1, lat_2):
    '''Apply an area crossing criterion.
    
    Larvae in ds selected while they are in a selected area.
    '''
    # particles are selected if they pass through given area.
    position =  ((lon_2 -lon_1) * (ds.lat - lat_1) - 
                     (ds.lon - lon_1) * (lat_2 - lat_1))
                        
    return position > 0.0, position < 0

#### identify tracks from Labrador sea or from Gulf Stream

In [ ]:
ds_lab_sea_in=[]
ds_60w_in=[]

for i in range(len(ds)):
    ds_in1, ds_notin1 = apply_left_of_line(ds[i],-75,-40,40,65)
    ds_in2, ds_notin2 = apply_left_of_line(ds[i],-95,-60,52,52)
    ds_lab_sea_in.append(ds_in1*ds_in2)

    ds_in, ds_notin = apply_left_of_line(ds[i],-60,-100,33,33)
    ds_60w_in.append(ds_in)


In [ ]:
ds_init_labsea=[]
for i in range(len(ds_init)):
    ds_init_labsea.append(ds_init[i].where(ds_lab_sea_in[i].max("obs")))
    
ds_init_60w=[]
for i in range(len(ds_init)):
    ds_init_60w.append(ds_init[i].where(ds_lab_sea_in[i].max("obs")==False).where(ds_60w_in[i].max("obs")))
    
ds_init_other=[]
for i in range(len(ds_init)):
    ds_init_other.append(ds_init[i].where(ds_60w_in[i].max("obs")==False).where(ds_lab_sea_in[i].max("obs")==False))
    
ds_lab_sea = []
for i in range(len(ds)):
    ds_lab_sea.append(ds[i].where(ds_lab_sea_in[i].max("obs")))

ds_60w = []
for i in range(len(ds)):
    ds_60w.append(ds[i].where(ds_lab_sea_in[i].max("obs")==False).where(ds_60w_in[i].max("obs")))
    
ds_other = []
for i in range(len(ds)):
    ds_other.append(ds[i].where(ds_lab_sea_in[i].max("obs")==False).where(ds_60w_in[i].max("obs")==False))   

## Transports

### Near-surface (0-500 m ) transport northwards across the OSNAP section in the Iceland Basin (30-20 W). Plotted by particle source

In [ ]:
def transports(dsinit):
    vt = []
    ht = []
    fwt = []
    tav = []
    sav = []
    ti = []
    for ds_1_init in dsinit:

        transport_1 = ds_1_init.u_normal.sum(dim='traj')*particle_section_area/1.0e06
        temperature_transport_1 = (ds_1_init.u_normal * ds_1_init.temp).sum(dim='traj')*particle_section_area/1.0e06
        salt_transport_1 = (ds_1_init.u_normal * ds_1_init.salt).sum(dim='traj')*particle_section_area/1.0e06

        vt.append(transport_1.data)
        ht.append(temperature_transport_1.data)
        fwt.append(salt_transport_1.data)
        tav.append(temperature_transport_1.data/transport_1.data)
        sav.append(salt_transport_1.data/transport_1.data)
        ti.append(ds_1_init.time.mean().data)

    ti = np.array(ti)
    
    x = np.argsort(ti)
    ti = ti[x]
    vt = np.array(vt)[x]
    ht = np.array(ht)[x]
    fwt = np.array(fwt)[x]
    tav = np.array(tav)[x]
    sav = np.array(sav)[x]
        
    return vt,ht,fwt,tav,sav,ti

In [ ]:
vt,ht,fwt,tav,sav,ti = transports(ds_init)

In [ ]:
vtls,htls,fwtls,tavls,savls,tils = transports(ds_init_labsea)

In [ ]:
vtgs,htgs,fwtgs,tavgs,savgs,tigs = transports(ds_init_60w)

In [ ]:
def plot_transports_by_source():
    sns.set(style="darkgrid")
    sns.set_palette("colorblind")
    fig,ax = plt.subplots(3,figsize = (8,9),sharex=True)
    ax[0].plot(ti,vt,label='total')
    ax[0].plot(tils,vtls,label='labrador sea')
    ax[0].plot(tigs,vtgs,label='gulf stream')
    ax[0].plot(ti,vt-vtls-vtgs,label='other')
    ax[0].legend(bbox_to_anchor=(1.0, 1.05))
    ax[0].set_ylabel('transport [Sv]')

    ax[1].plot(ti,ht,label='total')
    ax[1].plot(tils,htls,label='labrador sea')
    ax[1].plot(tigs,htgs,label='gulf stream')
    ax[1].plot(ti,ht-htls-htgs,label='other')
    ax[1].legend(bbox_to_anchor=(1.0, 1.05))
    ax[1].set_ylabel('temperature transport [Sv $\degree$C]')

    ax[2].plot(ti,fwt,label='total')
    ax[2].plot(tils,fwtls,label='labrador sea')
    ax[2].plot(tigs,fwtgs,label='gulf stream')
    ax[2].plot(ti,fwt-fwtls-fwtgs,label='other')
    ax[2].legend(bbox_to_anchor=(1.0, 1.05))
    ax[2].set_ylabel('salt transport [Sv PSU]')

Notice the larger difference between volume transports originating from the Gulf Stream and from the Labrador Sea in the period 2006-2011, and smaller difference after 2012.

In [ ]:
plot_transports_by_source()

In [ ]:
def plot_mean_properties_by_source():
    sns.set(style="darkgrid")
    sns.set_palette("colorblind")
    fig,ax = plt.subplots(2,figsize = (8,5),sharex=True)

    ax[0].plot(ti,tav,label='total')
    ax[0].plot(tils,tavls,label='labrador sea')
    ax[0].plot(tigs,tavgs,label='gulf stream')
    ax[0].legend(bbox_to_anchor=(1.0, 1.05))
    ax[0].set_ylabel('mean temperature [$\degree$C]')

    ax[1].plot(ti,sav,label='total')
    ax[1].plot(tils,savls,label='labrador sea')
    ax[1].plot(tigs,savgs,label='gulf stream')
    ax[1].legend(bbox_to_anchor=(1.0, 1.05))
    ax[1].set_ylabel('mean salinity [PSU]');



This is the average temperature and salinity when the water crosses the OSNAP section. So it will have been modified from the water properties in the 'source' region.

In [ ]:
plot_mean_properties_by_source()

### heat loss

In [ ]:
# ds_mask = ds

In [ ]:
ds_mask_lab_sea = []
for i in range(len(ds)):
    ds_mask_lab_sea.append(ds[i].where(ds[i].temp!=0.0).where(ds_lab_sea_in[i].max("obs")).dropna('traj', how='all').ffill(dim='obs'))

ds_mask_60w = []
for i in range(len(ds)):
    ds_mask_60w.append(ds[i].where(ds[i].temp!=0.0).where(ds_lab_sea_in[i].max("obs")==False).where(ds_60w_in[i].max("obs")).dropna('traj', how='all').ffill(dim='obs'))
    
ds_mask_other = []
for i in range(len(ds)):
    ds_mask_other.append(ds[i].where(ds[i].temp!=0.0).where(ds_lab_sea_in[i].max("obs")==False).where(ds_60w_in[i].max("obs")==False).dropna('traj', how='all').ffill(dim='obs'))

In [ ]:
ds_mask = []
for i in range(len(ds)):
    ds_mask.append(ds[i].where(ds[i].temp!=0.0).dropna('traj', how='all').ffill(dim='obs'))


In [ ]:
ds_mask[1]

In [ ]:
def plot_alongtrack_av_timeseries_chrono(dsi,dsmi):
    sns.set(style="darkgrid")
    sns.set_palette("Paired")

    fig,ax = plt.subplots(figsize = (13,8),sharex=True)

    for i in range(len(dsmi)):
        ax.plot(dsi[i].time[100],dsmi[i].temp.mean(dim='traj'))

    ax.set_ylim(2.5,18.5)

In [ ]:
def plot_alongtrack_av_flux_timeseries_chrono(dsi,dsmi):
    sns.set(style="darkgrid")
    sns.set_palette("Paired")

    fig,ax = plt.subplots(figsize = (13,8),sharex=True)

    for i in range(len(dsmi)):
        ax.plot(dsi[i].time[100],dsmi[i].temp_transport.sum(dim='traj')/dsmi[i].vol_transport.sum(dim='traj'))

    ax.set_ylim(2.5,18.5)

In [ ]:
def plot_alongtrack_av_timeseries_salt_chrono(dsi,dsmi):
    sns.set(style="darkgrid")
    sns.set_palette("Paired")

    fig,ax = plt.subplots(figsize = (13,8),sharex=True)

    for i in range(len(dsmi)):
        ax.plot(dsi[i].time[100],dsmi[i].salt.mean(dim='traj'))
    
    ax.set_ylim(34.41,36.15)

In [ ]:
def plot_alongtrack_av_flux_timeseries_salt_chrono(dsi,dsmi):
    sns.set(style="darkgrid")
    sns.set_palette("Paired")

    fig,ax = plt.subplots(figsize = (13,8),sharex=True)

    for i in range(len(dsmi)):
        ax.plot(dsi[i].time[100],dsmi[i].salt_transport.sum(dim='traj')/dsmi[i].vol_transport.sum(dim='traj'))

    ax.set_ylim(34.41,36.15)

## Mean temperature and salinity alongtrack.

Colours correspond to individualreleases. Particles were tracked backwards in time, so the right-hand end of each track section is the time it crossed OSNAP line.

The idea is to try to get an idea of how much the water properties have been modified between the source region and OSNAP.


Mean temperature, all tracks. Mostly just showing the seasonal cycle. Lower temperatures recorded in later years are recorded over the length of the track. This could be changing relative importance of sources, or changing properties at source.

In [ ]:
plot_alongtrack_av_timeseries_chrono(ds,ds_mask)

In [ ]:
plot_alongtrack_av_flux_timeseries_chrono(ds,ds_mask)

Mean temperature along track. Tracks from Labrador Sea.

In [ ]:
plot_alongtrack_av_timeseries_chrono(ds,ds_mask_lab_sea)

In [ ]:
plot_alongtrack_av_flux_timeseries_chrono(ds,ds_mask_lab_sea)

Mean temperature along track. Tracks from Gulf Stream.

In [ ]:
plot_alongtrack_av_timeseries_chrono(ds,ds_mask_60w)

In [ ]:
plot_alongtrack_av_flux_timeseries_chrono(ds,ds_mask_60w)

Mean temperature along track. Other tracks.

In [ ]:
plot_alongtrack_av_timeseries_chrono(ds,ds_mask_other)

In [ ]:
plot_alongtrack_av_flux_timeseries_chrono(ds,ds_mask_other)

Mean salinity, all tracks. Lower salinities recorded in later years are recorded over the length of the track. This could be changing sources, or changing properties at source.

In [ ]:
plot_alongtrack_av_timeseries_salt_chrono(ds,ds_mask)

In [ ]:
plot_alongtrack_av_flux_timeseries_salt_chrono(ds,ds_mask)

Mean salinity along track. Tracks from Labrador Sea.

In [ ]:
plot_alongtrack_av_timeseries_salt_chrono(ds,ds_mask_lab_sea)

In [ ]:
plot_alongtrack_av_flux_timeseries_salt_chrono(ds,ds_mask_lab_sea)

Mean salinity along track. Tracks from Gulf Stream.

In [ ]:
plot_alongtrack_av_timeseries_salt_chrono(ds,ds_mask_60w)

In [ ]:
plot_alongtrack_av_flux_timeseries_salt_chrono(ds,ds_mask_60w)

Mean salinity along track. Other tracks.

In [ ]:
plot_alongtrack_av_timeseries_salt_chrono(ds,ds_mask_other)

In [ ]:
plot_alongtrack_av_flux_timeseries_salt_chrono(ds,ds_mask_other)

### Some experimental plots

time and salinity when left Labrador Sea

In [ ]:
fig,ax = plt.subplots(figsize=(14,5))

for i in range(len(ds)):

    test = ds[i].where(ds_lab_sea_in[i].max("obs")).isel(obs=ds_lab_sea_in[i].argmax(dim='obs'),traj=ds_lab_sea_in[i].traj).dropna(dim='traj')
    pcm = ax.scatter(x=test.time.data,y=test.temp.data)


In [ ]:
fig,ax = plt.subplots(figsize=(14,5))

for i in range(len(ds)):

    test = ds[i].where(ds_lab_sea_in[i].max("obs")).isel(obs=ds_lab_sea_in[i].argmax(dim='obs'),traj=ds_lab_sea_in[i].traj).dropna(dim='traj')
    pcm = ax.scatter(x=test.time.data,y=test.salt.data)


time and salinity when left Labrador Sea

In [ ]:
fig,ax = plt.subplots(figsize=(14,5))

for i in range(len(ds)):

    test = ds[i].where((ds_lab_sea_in[i].max("obs")==False)).where(ds_60w_in[i].max("obs")).isel(obs=ds_60w_in[i].argmax(dim='obs'),traj=ds_60w_in[i].traj).dropna(dim='traj')
    pcm = ax.scatter(x=test.time.data,y=test.temp.data
                     ,c=test.salt.data,
              vmin=33,vmax=36,cmap=co.cm.haline
                    )
plt.colorbar(pcm,ax=ax,label = "salinity");

# cb = plt.colorbar(label = "date");


In [ ]:
# conda list

In [ ]:
ds0=ds[0]

In [ ]:
ds0

In [ ]:
ds_init0 = ds_init[0]

In [ ]:
ds_init0

In [ ]:
ds0.temp*ds_init0.u_normal